In [9]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
load_dotenv()

groq_api_key = os.getenv('GROQ_API_KEY')

In [10]:
model=ChatGroq(model='Gemma2-9b-It', groq_api_key=groq_api_key)
model


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000022DA6BAE670>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000022DA6BB1220>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [11]:
from langchain_core.messages import HumanMessage, AIMessage
response1=model.invoke([HumanMessage(content="Hi, This is Yogesh an AI Engineer at MasterCard")])
response1=response1.content

In [12]:
response2 = model.invoke(
    [HumanMessage(content="Hi, This is Yogesh an AI Engineer at MasterCard"),
    AIMessage(content=response1),
    HumanMessage(content="Hi, What is my name and do I do?")
    ])
response2

AIMessage(content="You told me your name is Yogesh and that you are an AI Engineer at MasterCard. 😊 \n\nIs there anything else you'd like to know or discuss?  I'm here to help!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 91, 'total_tokens': 136, 'completion_time': 0.081818182, 'prompt_time': 0.002678362, 'queue_time': 0.021169788, 'total_time': 0.084496544}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-e4ed0d4e-08d6-4a9e-a1ca-4b4c5c9855b8-0', usage_metadata={'input_tokens': 91, 'output_tokens': 45, 'total_tokens': 136})

In [13]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_id(session_id: str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_id)


In [ ]:
config={"session_id": "chat1"}
with_message_history.invoke(
    [HumanMessage(content="Hi, This is Yogesh an AI Engineer at MasterCard")],
    config=config
)

AIMessage(content="Hello Yogesh, it's nice to meet you! \n\nAs an AI, I don't have personal experiences or affiliations like being an engineer at MasterCard.  \n\nWhat can I do for you today? Perhaps you have some questions about AI, need help with a coding task, or want to discuss the latest advancements in the field?  I'm here to help in any way I can. 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 89, 'prompt_tokens': 20, 'total_tokens': 109, 'completion_time': 0.161818182, 'prompt_time': 9.883e-05, 'queue_time': 0.014912108, 'total_time': 0.161917012}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-6939d87d-447c-4907-a732-2199601fee44-0', usage_metadata={'input_tokens': 20, 'output_tokens': 89, 'total_tokens': 109})

In [15]:
with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config)

AIMessage(content='You told me your name is Yogesh. 😊  \n\nIs there anything else I can help you with?  \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 121, 'total_tokens': 147, 'completion_time': 0.047272727, 'prompt_time': 0.003678948, 'queue_time': 0.01131469, 'total_time': 0.050951675}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-d0cb5e14-85b1-4be6-bb0b-2d1e57e6bcb5-0', usage_metadata={'input_tokens': 121, 'output_tokens': 26, 'total_tokens': 147})

In [17]:
# change the session id using new config 
config2= {"session_id":"chat2"}
response2=with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config2)
response2.content

"I still don't know your name! \n\nYou'll have to tell me.  What's your name?  😄  \n"

## ChatBot with prompt template

In [28]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system", "You are helpful assistant. Answer questions best of your knowledge"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain = prompt|model

In [29]:
chain.invoke({"messages":[
    HumanMessage(content="Hi I'm Yogesh")
]})

AIMessage(content="Hello Yogesh! 👋 \n\nIt's nice to meet you.  \n\nHow can I help you today? 😊  I'm ready for any questions you have, or if you'd like to chat about something, I'm all ears! 👂 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 26, 'total_tokens': 85, 'completion_time': 0.107272727, 'prompt_time': 0.000220929, 'queue_time': 0.013311309, 'total_time': 0.107493656}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-b120b14b-e974-4e08-a318-3e3736df83f3-0', usage_metadata={'input_tokens': 26, 'output_tokens': 59, 'total_tokens': 85})

In [33]:
with_messge_history=RunnableWithMessageHistory(chain, get_session_id)
config3={"session_id": "chat3"}
response=with_messge_history.invoke(
    HumanMessage(content="Hi I am yogesh"), config=config3
)
response.content


"Hi Yogesh! 👋\n\nIt's great to meet you.  \n\nIs there anything you'd like to ask me or have me do? I'm here to help! 😊 \n\n\n"

In [38]:
# Add more complexit adding one more input

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer in {language}"),
    MessagesPlaceholder(variable_name="messages")]
    )

chain = prompt|model


In [41]:

response = chain.invoke({"messages":[HumanMessage(content="HI I am Yogesh")],
                        "language": "Marathi"})
response.content

'नमस्कार, योगेश! मी तुमच्या मदतीसाठी येथे आहे. काय मदत हवी आहे? 😊  \n'

In [45]:
with_message_history=RunnableWithMessageHistory(
    chain, get_session_id,
    input_messages_key="messages"
)

config4 = {"session_id": "chat4"}

response = with_message_history.invoke({"messages":[HumanMessage(content="HI I am Yogesh")],                      
        "language": "Marathi"}, config=config4)
response.content

'नमस्कार योगेश, मी तुमचा मदत करण्यासाठी येथे आहे. तुम्हाला काय मदत हवी आहे? \n'